In [ ]:
import os
import radiomics
import pandas as pd
import numpy as np
import SimpleITK as sitk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
para_path_CT = "./Params.yaml"
extractor_CT = radiomics.featureextractor.RadiomicsFeatureExtractor(para_path_CT)
df_CT = pd.DataFrame()

In [ ]:
path = '../data'
for i in os.listdir(path):
    folder_a = os.listdir(os.path.join(path,i))
    for folder in folder_a:
        print(folder)
        df_CT_ID = pd.DataFrame([str(folder)], columns =["ID"])
        label = pd.DataFrame([1],columns=["label"])
        imagePath = os.path.join(path,i,folder,'CT_low.nii.gz')
        maskPath = CT_M_path = os.path.join('../data',folder,'CT_M.nii.gz')
        mask = sitk.ReadImage(maskPath)
        mask_pixel = sitk.GetArrayFromImage(mask)
        featureVector_CT = extractor_CT.execute(imagePath, maskPath)  
        df_add_CT = pd.DataFrame.from_dict(featureVector_CT.values()).T
        df_add_CT.columns = featureVector_CT.keys()
        col1 = pd.concat([df_CT_ID,label],axis=1)
        col2 = pd.concat([col1,df_add_CT],axis=1)
        df_CT = pd.concat([df_CT, col2])
        
        

In [ ]:
df_CT.to_csv("./extract_CT.csv")

In [ ]:
para_path = "./Params_2.yaml"
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(para_path)
df_PET = pd.DataFrame()


In [ ]:
for i in os.listdir(path):
    folder_a = os.listdir(os.path.join(path,i))
    for folder in folder_a:
        print(folder)
        df_PET_ID = pd.DataFrame([str(folder)], columns =["ID"])
        label = pd.DataFrame([1],columns=["label"])
        imagePath = os.path.join(path,'input',i,folder,'PT_0000.nii.gz')
        maskPath = os.path.join(path,'output',i,folder,'PT_FDG_Tumor_PT_0000.nii.gz')
        mask = sitk.ReadImage(maskPath)
        mask_pixel = sitk.GetArrayFromImage(mask)
        featureVector = extractor.execute(imagePath, maskPath) 
        df_add = pd.DataFrame.from_dict(featureVector.values()).T
        df_add.columns = featureVector.keys()
        col1 = pd.concat([df_PET_ID,label],axis=1)
        col2 = pd.concat([col1,df_add],axis=1)
        df_PET = pd.concat([df_PET, col2])
            
columns_list = df_PET.columns.to_list()
pet_columns = ['PET_'+str(i) for i in columns_list]
df_PET.columns = pet_columns


In [ ]:
df_PET = df_PET.rename(columns={'PET_ID':'ID'})
df_PET.to_csv("./extract_SUV.csv")

In [ ]:
xlsx2_filepath = "./extract_CT.csv"
xlsx3_filepath = "./extract_SUV.csv"

data2_nnor = pd.read_csv(xlsx2_filepath)
data3_nnor = pd.read_csv(xlsx3_filepath)

In [ ]:
import pandas as pd
import numpy as np
ex_vali_fp = './features_non.csv'
nnor = pd.read_csv(ex_vali_fp, encoding='gbk')
X_nnor = nnor.loc[:,"AGE":]
colNames = X_nnor.columns
X_nnor = X_nnor.astype(np.float64)
X2 = X_nnor.fillna(X_nnor.mean())
X2 = StandardScaler().fit_transform(X_nnor)
X2 = pd.DataFrame(X2)
X2.columns = colNames
data2 = pd.concat([nnor.loc[:,"label":'lobulation'],X2],axis=1)
data2.to_csv("./features.csv")
data2.head(5)

In [ ]:
import pandas as pd
data = pd.read_csv("./SUV/features.csv")



In [ ]:
y = data["label"]
X = data.loc[:,"Gender":]

In [ ]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)
validation = pd.concat([X_test,y_test],axis=1)
train_test = pd.concat([X_train,y_train],axis=1)
# print(y_test)

In [ ]:
validation.to_csv("./validation.csv")
train_test.to_csv("./train_test.csv")